In [4]:
import pandas as pd
import numpy as np
import folium

from geopy import Nominatim
from geopy.distance import great_circle as GC

from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from pyroutelib3 import Router

# 1) Query Coordinates:

In [5]:
# Define list of target points addresses:
addresses = [
    'Sternstraße 12 Münster',
    'Bohlweg 17 Münster',
    'Scharnhorststraße 85 Münster',
    'Geiststraße 46 Münster',
    'Kanalstraße 14 Münster',

    'Junkerstraße 2 Münster',
    'Alter Fischmarkt 11 Münster',
    'Scharnhorststraße 2 Münster',
    'Königsstraße 3 Münster',
    'Fürstenbergstraße 30 Münster',
]

In [6]:
# Get coordinates and create list of target points including address and coordinates retrieved from Open Street Map Server:
geolocator = Nominatim(user_agent="TSP_Muenster")

addresses_geo_data = []
for address in addresses:

    loc = geolocator.geocode(address)

    # Handle cases in which address couldn't be retrieved:
    if loc is None:
        print(f"Address couldn't be located: {address}")
        break

    addresses_geo_data.append({
        'address': address,
        'coords': (loc.latitude,loc.longitude)
    })

# Print results:
addresses_geo_data

[{'address': 'Sternstraße 12 Münster',
  'coords': (51.95797245, 7.641767157780192)},
 {'address': 'Bohlweg 17 Münster',
  'coords': (51.966913399999996, 7.639723184318184)},
 {'address': 'Scharnhorststraße 85 Münster',
  'coords': (51.95102425, 7.610322795770778)},
 {'address': 'Geiststraße 46 Münster',
  'coords': (51.94913765, 7.621511030738704)},
 {'address': 'Kanalstraße 14 Münster',
  'coords': (51.96902075, 7.631288845058138)},
 {'address': 'Junkerstraße 2 Münster', 'coords': (51.9536747, 7.6298298)},
 {'address': 'Alter Fischmarkt 11 Münster', 'coords': (51.9639051, 7.6296169)},
 {'address': 'Scharnhorststraße 2 Münster',
  'coords': (51.9554004, 7.619202070882074)},
 {'address': 'Königsstraße 3 Münster', 'coords': (51.957206, 7.6266662)},
 {'address': 'Fürstenbergstraße 30 Münster',
  'coords': (51.9621944, 7.6353705)}]

In [8]:
# Create Warehouse:
warehouse_address = 'Am Mittelhafen 14 Münster'
loc = geolocator.geocode(warehouse_address)

warehouse = {
    'address': warehouse_address,
    'coords': (loc.latitude,loc.longitude)
}

# 2) Visualization
## Only Locations

In [9]:
# Create map object:
map_osm = folium.Map(location=warehouse['coords'], zoom_start=14, tiles='Open Street Map')

# Create icon for warehouse:
folium.Marker(
    location=warehouse['coords'],
    icon=folium.Icon(color='green', icon='industry', prefix='fa'),
    popup=warehouse['address'],
    tooltip=warehouse["address"],
    draggable=False).add_to(map_osm)

# Create icons for each address:
for address in addresses_geo_data:
    folium.Marker(
        location=address['coords'],
        icon=folium.Icon(color='darkblue', icon='home'),
        popup=address['address'],
        tooltip=address["address"],
        draggable=False).add_to(map_osm)

map_osm

# 3) Distance Matrix:

In [7]:
def get_distance(point1: tuple, point2: tuple) -> float:

    # Air distance in meters:
    dist = GC(point1, point2).km * 1000

    return round(dist,2)


def create_distance_matrix(coords: list, address_name:list, format='DataFrame', verbose=False):
    # Create empty np-array:
    dist_array = np.empty((len(coords),len(coords)))

    # Compute distances:
    for i in range(0,len(coords)):
        for j in range(i,len(coords)):
            if i < j:
                dist = get_distance(coords[i],coords[j])

                if verbose:
                    print(f"Distance between: {address_name[i]} and {address_name[j]}: {dist} km")

                # Assuming symmetric TSP:
                dist_array[i][j] = dist
                dist_array[j][i] = dist

            elif i == j:
                dist_array[i][i] = 0

            else:
                continue

    if format == 'NumpyArray':
        return dist_array

    else:
        # Create pandas distance matrix:
        return pd.DataFrame(data=dist_array, index=address_name, columns=address_name)

In [8]:
# Create list of addresses including warehouse and target points to be used when creating the distance matrix dataframe as column names and index names:
address_list = [warehouse['address']] + [obj['address'] for obj in addresses_geo_data]
coords_list = [warehouse['coords']] + [obj['coords'] for obj in addresses_geo_data]

# Call function twice for returning a numpyArray and a DataFrame:
dist_matrix_np = create_distance_matrix(address_name=address_list, coords=coords_list, verbose=False, format='NumpyArray')
dist_matrix_df = create_distance_matrix(address_name=address_list, coords=coords_list, verbose=False, format='DataFrame')

In [9]:
dist_matrix_df

,Hafenweg 1 Münster,Sternstraße 12 Münster,Bohlweg 17 Münster,Scharnhorststraße 85 Münster,Geiststraße 46 Münster,Kanalstraße 14 Münster,Junkerstraße 2 Münster,Alter Fischmarkt 11 Münster,Scharnhorststraße 2 Münster,Königsstraße 3 Münster,Fürstenbergstraße 30 Münster
Hafenweg 1 Münster,0.00,614.75,1611.06,2123.43,1400.18,1963.79,792.30,1501.14,1544.03,1128.67,1154.86
Sternstraße 12 Münster,614.75,0.00,1004.00,2289.14,1700.60,1422.90,947.38,1062.20,1572.49,1038.27,642.26
Bohlweg 17 Münster,1611.06,1004.00,0.00,2679.55,2337.57,623.52,1620.66,768.97,1901.55,1401.96,603.55
Scharnhorststraße 85 Münster,2123.43,2289.14,2679.55,0.00,794.96,2463.39,1368.94,1949.20,779.14,1314.11,2118.66
Geiststraße 46 Münster,1400.18,1700.60,2337.57,794.96,0.00,2310.19,761.28,1733.47,714.14,964.21,1734.90
Kanalstraße 14 Münster,1963.79,1422.90,623.52,2463.39,2310.19,0.00,1709.33,580.25,1726.14,1351.38,808.93
Junkerstraße 2 Münster,792.30,947.38,1620.66,1368.94,761.28,1709.33,0.00,1137.66,753.15,448.53,1020.60
Alter Fischmarkt 11 Münster,1501.14,1062.20,768.97,1949.20,1733.47,580.25,1137.66,0.00,1184.73,771.86,437.70
Scharnhorststraße 2 Münster,1544.03,1572.49,1901.55,779.14,714.14,1726.14,753.15,1184.73,0.00,549.48,1340.94
Königsstraße 3 Münster,1128.67,1038.27,1401.96,1314.11,964.21,1351.38,448.53,771.86,549.48,0.00,814.49


# 4) Traveling Salesman Problem

In [10]:
# The index manager handles the conversion of the solver's internal indices to the location indices of our distance matrix:
index_manager = pywrapcp.RoutingIndexManager(len(dist_matrix_np), 1, 0) ## 11 nodes, 1 vehicle, 0 warehouse_index

# The routing model is the central object that we can configure to solve our problem:
routing_model = pywrapcp.RoutingModel(index_manager)

In [11]:
# Function that returns the distance between two points. In this case the easiest solution is to use the distance matrix:
def distance_callback(from_index, to_index):
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = index_manager.IndexToNode(from_index)
    to_node = index_manager.IndexToNode(to_index)
    return dist_matrix_np[from_node][to_node]

# Create Callback that is needed to connect our routing model object with the distance matrix:
transit_callback_index = routing_model.RegisterTransitCallback(distance_callback)

# The Arc Cost Evaluator tells the model how to compute the costs of choosing one arc and thus driving from one point A to another point B:
routing_model.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Set the search strategy to the default strategy:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

# Use the 'greedy approach' to create an initial solution that is then improved:
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [12]:
# Function to print the current solution to the console:
def print_solution(manager, routing, solution):
    index = routing.Start(0)
    plan_output = 'Route for first vehicle:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f"{manager.IndexToNode(index)} -> "
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f"{manager.IndexToNode(index)}\n"
    plan_output += f"Route distance: {route_distance} m\n"
    print(plan_output)

# Compute solution:
solution = routing_model.SolveWithParameters(search_parameters)

if solution:
    print_solution(index_manager, routing_model, solution)

Route for first vehicle:
0 -> 1 -> 10 -> 2 -> 5 -> 7 -> 9 -> 8 -> 3 -> 4 -> 6 -> 0
Route distance: 7508 m



In [13]:
def get_tours(solution, routing, manager):
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    all_tours = []
    for vehicle in range(routing.vehicles()):
        index = routing.Start(vehicle)
        curr_tour = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            curr_tour.append(manager.IndexToNode(index))
        all_tours.append(curr_tour)
    return all_tours

tour_plan = get_tours(solution, routing_model, index_manager)

In [14]:
tour_plan

[[0, 1, 10, 2, 5, 7, 9, 8, 3, 4, 6, 0]]

# 5) Visualize Results
## One Vehicle - Full tour - Lines between points

In [15]:
## Create map object:
map_osm = folium.Map(location=warehouse['coords'], zoom_start=14, tiles='Open Street Map')

## Create icon for warehouse:
folium.Marker(
    location=warehouse['coords'],
    icon=folium.Icon(color='green', icon='industry', prefix='fa'),
    popup=warehouse['address'],
    tooltip=warehouse["address"],
    draggable=False).add_to(map_osm)

## Create icons for each address:
for address in addresses_geo_data:
    folium.Marker(
        location=address['coords'],
        icon=folium.Icon(color='darkblue', icon='home'),
        popup=address['address'],
        tooltip=address["address"],
        draggable=False).add_to(map_osm)

## Create connections between target points:
# Iterate over tour plan:
for tour in tour_plan:
    # Iterate from first to penultimate element:
    for i in range(0,len(tour) - 1):
        coords_point_a = coords_list[tour[i]]
        coords_point_b = coords_list[tour[i+1]]

        folium.PolyLine([coords_point_a, coords_point_b],
                        color="black",
                        weight=3).add_to(map_osm)

map_osm

# 6) Compute route between points:

In [16]:
tour_plan

[[0, 1, 10, 2, 5, 7, 9, 8, 3, 4, 6, 0]]

In [17]:
# Initialize router:
router = Router("foot")

# Stores the routes of all tours:
tour_plan_all_routes = []

# Iterate over tours:
for tour in tour_plan:

    # Stores the routes of the current tour:
    curr_tour_route = []

    # Pairwise iterate to obtain paths between two given points:
    for idx in range(0, len(tour) - 1):

        curr_start_point = coords_list[tour[idx]]
        curr_end_point = coords_list[tour[idx+1]]

        # Find Start and End Nodes near desired location:
        start = router.findNode(*curr_start_point)
        end = router.findNode(*curr_end_point)

        # Get route:
        status, route = router.doRoute(start, end)

        # Get coordinates of route:
        if status == 'success':
            routeLatLons = list(map(router.nodeLatLon, route))
            curr_tour_route.append(routeLatLons)

    tour_plan_all_routes.append(curr_tour_route)

In [18]:
tour_plan_all_routes

[[[(51.9524542, 7.6412202),
   (51.952475, 7.6415646),
   (51.9528429, 7.641595),
   (51.9531457, 7.6416092),
   (51.9532211, 7.6416116),
   (51.9533084, 7.6416153),
   (51.9533284, 7.6416175),
   (51.9533728, 7.6416225),
   (51.9534408, 7.6416276),
   (51.9541937, 7.6416838),
   (51.9544655, 7.6417012),
   (51.9544991, 7.6416914),
   (51.9545347, 7.6416661),
   (51.9545503, 7.6415893),
   (51.9545549, 7.6415367),
   (51.9545813, 7.6415685),
   (51.9546049, 7.6415795),
   (51.9546375, 7.6416),
   (51.9546914, 7.6416242),
   (51.9547034, 7.6416353),
   (51.9547532, 7.6416525),
   (51.954825, 7.6416693),
   (51.9548856, 7.6416691),
   (51.9551079, 7.6416929),
   (51.9551585, 7.6417176),
   (51.9552161, 7.6417457),
   (51.9553458, 7.6417604),
   (51.9558163, 7.6417887),
   (51.9558784, 7.6418389),
   (51.9559106, 7.6418726),
   (51.9561514, 7.6421303),
   (51.9562595, 7.6422459),
   (51.9565273, 7.6425184),
   (51.9565745, 7.642583),
   (51.9566223, 7.6426498),
   (51.9569078, 7.6419072),

# 7) Visualization
## One Vehicle - Full tour including exact routes

In [19]:
## Create map object:
map_osm = folium.Map(location=warehouse['coords'], zoom_start=14, tiles='Open Street Map')

## Create icon for warehouse:
folium.Marker(
    location=warehouse['coords'],
    icon=folium.Icon(color='green', icon='industry', prefix='fa'),
    popup=warehouse['address'],
    tooltip=warehouse["address"],
    draggable=False).add_to(map_osm)

## Create icons for each address:
for address in addresses_geo_data:
    folium.Marker(
        location=address['coords'],
        icon=folium.Icon(color='darkblue', icon='home'),
        popup=address['address'],
        tooltip=address["address"],
        draggable=False).add_to(map_osm)

## Create connections between target points:
# Iterate over tour plan:
for tour in tour_plan_all_routes:
    # Iterate over each connection from one point to the next point:
    for route in tour:
        # Iterate from first to penultimate element:
        for i in range(0,len(route) - 1):
            coords_point_a = route[i]
            coords_point_b = route[i+1]

            folium.PolyLine([coords_point_a, coords_point_b],
                            color="black",
                            weight=3).add_to(map_osm)

map_osm

# 8) Extension to Multiple Vehicles and Capacity Constraints:

In [57]:
## New:
# Extend to multiple vehicles:
num_vehicles = 3

# Capacity constraints:
demands=[0,20,40,50,30,40,40,60,40,20,20]
vehicle_capacities=[200, 100, 100]

# The index manager handles the conversion of the solver's internal indices to the location indices of our distance matrix:
index_manager = pywrapcp.RoutingIndexManager(len(dist_matrix_np), num_vehicles, 0) ## 11 nodes, 1 vehicle, 0 warehouse_index


## Old:
# The routing model is the central object that we can configure to solve our problem:
routing_model = pywrapcp.RoutingModel(index_manager)

# Create Callback that is needed to connect our routing model object with the distance matrix:
transit_callback_index = routing_model.RegisterTransitCallback(distance_callback)

# The Arc Cost Evaluator tells the model how to compute the costs of choosing one arc and thus driving from one point A to another point B:
routing_model.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Set the search strategy to the default strategy:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

# Use the 'greedy approach' to create an initial solution that is then improved:
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


## New:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of a node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = index_manager.IndexToNode(from_index)
    return demands[from_node]

demand_callback_index = routing_model.RegisterUnaryTransitCallback(demand_callback)

routing_model.AddDimensionWithVehicleCapacity(
    demand_callback_index,  # evaluator_index
    0,  # slack_max
    vehicle_capacities,  # vehicle maximum capacities
    True,  # start cumulative to zero
    'Capacity' # name
)

True

In [58]:
# Function to print the current solution to the console:
def print_solution(manager, routing, solution):

    for vehicle_id in range(num_vehicles):

        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += demands[node_index]
            plan_output += f"{manager.IndexToNode(index)} Load ({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Route distance: {route_distance} m\n"
        print(plan_output)

# Compute solution:
solution = routing_model.SolveWithParameters(search_parameters)

if solution:
    print_solution(index_manager, routing_model, solution)

Route for vehicle 0:
0 Load (0) -> 10 Load (20) -> 7 Load (80) -> 5 Load (120) -> 2 Load (160) -> 1 Load (180) -> 0
Route distance: 4412 m

Route for vehicle 1:
0 Load (0) -> 4 Load (30) -> 3 Load (80) -> 0
Route distance: 4317 m

Route for vehicle 2:
0 Load (0) -> 6 Load (40) -> 8 Load (80) -> 9 Load (100) -> 0
Route distance: 3222 m



# 9) Visualization
## Multiple Vehicles - Lines between points

In [59]:
# Get Tour Plan:
extended_tour_plan = get_tours(solution, routing_model, index_manager)
extended_tour_plan

[[0, 10, 7, 5, 2, 1, 0], [0, 4, 3, 0], [0, 6, 8, 9, 0]]

In [60]:
## Create map object:
map_osm = folium.Map(location=warehouse['coords'], zoom_start=14, tiles='Open Street Map')

## Create icon for warehouse:
folium.Marker(
    location=warehouse['coords'],
    icon=folium.Icon(color='green', icon='industry', prefix='fa'),
    popup=warehouse['address'],
    tooltip=warehouse["address"],
    draggable=False).add_to(map_osm)

## Create icons for each address:
# Get list of available colors:
color_lst = list(folium.map.Icon.color_options)
color_lst = ['darkblue', 'darkred', 'darkpurple']

# Iterate over tours:
for tour_num, tour in enumerate(extended_tour_plan):
    # Iterate over single tour:
    for address_idx in tour:
        # Skip warehouse address:
        if address_idx == 0:
            continue

        # Create marker:
        folium.Marker(
            location=coords_list[address_idx],
            icon=folium.Icon(color=color_lst[tour_num], icon='home'),
            popup=address_list[address_idx],
            tooltip=address_list[address_idx],
            draggable=False).add_to(map_osm)

## Create connections between target points:
# Iterate over tour plan:
for tour_num, tour in enumerate(extended_tour_plan):
    # Iterate from first to penultimate element:
    for i in range(0,len(tour) - 1):
        coords_point_a = coords_list[tour[i]]
        coords_point_b = coords_list[tour[i+1]]

        folium.PolyLine([coords_point_a, coords_point_b],
                        color="black",
                        weight=3).add_to(map_osm)

map_osm

## Multiple Vehicles - Routes between points

In [61]:
# Initialize router:
router = Router("foot")

# Stores the routes of all tours:
tour_plan_all_routes = []

# Iterate over tours:
for tour in extended_tour_plan:

    # Stores the routes of the current tour:
    curr_tour_route = []

    # Pairwise iterate to obtain paths between two given points:
    for idx in range(0, len(tour) - 1):

        curr_start_point = coords_list[tour[idx]]
        curr_end_point = coords_list[tour[idx+1]]

        # Find Start and End Nodes near desired location:
        start = router.findNode(*curr_start_point)
        end = router.findNode(*curr_end_point)

        # Get route:
        status, route = router.doRoute(start, end)

        # Get coordinates of route:
        if status == 'success':
            routeLatLons = list(map(router.nodeLatLon, route))
            curr_tour_route.append(routeLatLons)

    tour_plan_all_routes.append(curr_tour_route)

In [76]:
## Create map object:
map_osm = folium.Map(location=warehouse['coords'], zoom_start=14, tiles='Open Street Map')

## Create icon for warehouse:
folium.Marker(
    location=warehouse['coords'],
    icon=folium.Icon(color='green', icon='industry', prefix='fa'),
    popup=warehouse['address'],
    tooltip=warehouse["address"],
    draggable=False).add_to(map_osm)

## Create icons for each address:
# Get list of available colors:
color_lst = list(folium.map.Icon.color_options)
color_lst = ['darkblue', 'darkred', 'darkpurple']
color_lst2 = ['darkblue', 'darkred', 'purple']

# Iterate over tours:
for tour_num, tour in enumerate(extended_tour_plan):
    # Iterate over single tour:
    for address_idx in tour:
        # Skip warehouse address:
        if address_idx == 0:
            continue

        # Create marker:
        folium.Marker(
            location=coords_list[address_idx],
            icon=folium.Icon(color=color_lst[tour_num], icon='home'),
            popup=address_list[address_idx],
            tooltip=address_list[address_idx],
            draggable=False).add_to(map_osm)

## Create connections between target points:
# Iterate over tour plan:
for tour_num, tour in enumerate(tour_plan_all_routes):
    # Iterate over each connection from one point to the next point:
    for route in tour:
        # Iterate from first to penultimate element:
        for i in range(0,len(route) - 1):
            coords_point_a = route[i]
            coords_point_b = route[i+1]

            folium.PolyLine([coords_point_a, coords_point_b],
                             color=color_lst2[tour_num],
                             weight=3).add_to(map_osm)

map_osm

In [ ]:
## Create map object:
map_osm = folium.Map(location=warehouse['coords'], zoom_start=14, tiles='Open Street Map')

## Create icon for warehouse:
folium.Marker(
    location=warehouse['coords'],
    icon=folium.Icon(color='green', icon='industry', prefix='fa'),
    popup=warehouse['address'],
    tooltip=warehouse["address"],
    draggable=False).add_to(map_osm)